In [1]:
from sklearn import preprocessing
from sklearn import feature_extraction, model_selection, naive_bayes, metrics, svm
import itertools
from eval_utils import evaluate
import pandas as pd
import numpy as np
import os

In [20]:
with open(os.path.join(os.getcwd(), '../Data/contents.csv'), 'r+') as f:
    d = pd.read_csv(f)

In [22]:
d = d[['label', 'filter']]

In [26]:
filter_na = d.loc[~d['label'].isna()]

In [27]:
filter_na

,label,filter
0,1.0,Sun Received id Sun Received Sun Subject Gener...
1,0.0,Sun Received id Sun Received Postfix id Sun Re...
2,1.0,Sun Received id Sun Received id Sun Subject au...
3,1.0,Sun Received id Sun Received normative id corn...
4,1.0,Sun Received id Sun Received id Sun Subject tr...
...,...,...
75417,1.0,Received id Date Received id Car Subject reply...
75418,1.0,Received id Date Received id Car Subject reply...
75419,0.0,Received id Received id Received id Received i...
75420,0.0,Received id Received id Received id Received n...


In [28]:
# One hot encode the email to sparse matrices
f = feature_extraction.text.CountVectorizer(stop_words = 'english')
X = f.fit_transform(filter_na['filter'])
np.shape(X)

(75419, 32922)

In [6]:
print(f.get_feature_names())

['aa', 'aal', 'aam', 'aardvark', 'aba', 'abaca', 'abacate', 'aback', 'abacus', 'abalone', 'abandon', 'abandoned', 'abandoner', 'abandonment', 'abas', 'abase', 'abash', 'abashed', 'abate', 'abatement', 'abater', 'abb', 'abbas', 'abbess', 'abbey', 'abbot', 'abbreviate', 'abbreviation', 'abdicate', 'abdication', 'abdomen', 'abdominal', 'abduction', 'abductor', 'abear', 'abecedarian', 'abed', 'aberrant', 'aberrate', 'aberration', 'aberrational', 'abet', 'abeyance', 'abhor', 'abhorrent', 'abide', 'abiding', 'abigail', 'ability', 'abject', 'ablation', 'ablative', 'ablaze', 'able', 'ablest', 'ablow', 'ablution', 'ably', 'abnormal', 'abnormally', 'aboard', 'abode', 'abolish', 'abolishment', 'abolition', 'abolitionism', 'abolitionist', 'abominable', 'abominably', 'abomination', 'abord', 'aboriginal', 'abort', 'aborted', 'abortion', 'abortive', 'abortively', 'abound', 'abounding', 'abouts', 'aboveground', 'abracadabra', 'abrade', 'abrasion', 'abrasive', 'abreact', 'abreast', 'abret', 'abridge', 

In [29]:
# Split train-test set
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, filter_na['label'], test_size=0.33, random_state=42)
print("Data shape for training: {}\nData shape for testing: {}".format(np.shape(X_train), np.shape(X_test)))

Data shape for training: (50530, 32922)
Data shape for testing: (24889, 32922)


In [34]:
spam_proba = len(y_train[y_train == 1])/len(y_train)
#Create a Gaussian Classifier
model = naive_bayes.GaussianNB()
# Train the model using the training sets
%timeit model.fit(X_train.toarray(),y_train)
%timeit pred = model.predict(X_test.toarray())
evaluate(pred, y_test)

2min 53s ± 6.42 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
55 s ± 5.01 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
Precision: 0.9759297801103972		 Recall: 0.6541517559289137		 F1 Score: 0.7832812840438667		 Accuracy: 0.7602153561814456


In [35]:
modelM = naive_bayes.MultinomialNB()
%timeit modelM.fit(X_train, y_train)
%timeit predM = modelM.predict(X_test)
evaluate(predM, y_test)

36.6 ms ± 5.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
10.1 ms ± 314 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Precision: 0.996181055531209		 Recall: 0.9651240371201553		 F1 Score: 0.9804066543438078		 Accuracy: 0.9744465426493631


In [36]:
modelC = naive_bayes.ComplementNB()
%timeit modelC.fit(X_train, y_train)
%timeit predC = modelC.predict(X_test)
evaluate(predC, y_test)

30.8 ms ± 844 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
9.18 ms ± 200 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Precision: 0.9963581564736909		 Recall: 0.9624552677867411		 F1 Score: 0.9791133187301393		 Accuracy: 0.9727992285748724


In [37]:
modelS = svm.LinearSVC()
%timeit modelS.fit(X_train, y_train)
%timeit predS = modelS.predict(X_test)
evaluate(predS, y_test)

/Users/tranle/anaconda3/envs/IR/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


1.25 s ± 51.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.11 ms ± 145 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Precision: 0.9967256851806937		 Recall: 0.9970279614241524		 F1 Score: 0.9968768003881259		 Accuracy: 0.9958616256177428


In [19]:
res1 = predS - y_test

In [20]:
len(res1[res1 != 0])

252